In [1]:
source="""
#include <metal_stdlib>
using namespace metal;

kernel void monte_carlo(const device float2 *inPoints [[buffer(0)]],
                        device bool *outBools [[buffer(1)]],
                        uint id [[thread_position_in_grid]]) {
  const float2 location = inPoints[id];
  const float x = location.x;
  const float y = location.y;
  outBools[id] = (sqrt((x * x) + (y * y)) < 1.0) ? true : false;
}"""

In [2]:
import runmetal
import numpy

In [3]:
pm=runmetal.PyMetal()

In [4]:
pm.opendevice()
pm.openlibrary(source)

In [5]:
fn=pm.getfn("monte_carlo")

In [6]:
fn

<_MTLFunctionInternal: 0x7ff3f83c37d0>
    name = monte_carlo 
    device = <MTLIGAccelDevice: 0x7ff3f61ef800>
        name = Intel Iris Graphics 
    functionType = MTLFunctionTypeKernel 
    attributes: <none>

In [7]:
ibuf=pm.numpybuffer(numpy.float32(numpy.random.random(2*1024*1024)))
obuf=pm.emptybuffer(1*1024*1024)

In [8]:
cqueue, cbuffer=pm.getqueue()

In [9]:
pm.enqueue_compute(cbuffer, fn, [ibuf, obuf])
pm.enqueue_blit(cbuffer, obuf)

In [10]:
pm.start_process(cbuffer)
pm.wait_process(cbuffer)

In [11]:
result=pm.buf2numpy(obuf, dtype=numpy.bool8)

In [12]:
numpy.sum(result==True)*4/1024/1024

3.14056396484375